In [1]:
import pandas as pd
import Bio
from Bio import SeqIO

In [2]:
thermo_gff = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_July_2023/OORB003_TnSeq/for_analysis/modified_gff_file_finalized_08_08_2023'


In [4]:
thermo = pd.read_csv(thermo_gff, sep='\t', header=None, comment='#')

thermo.head()

,0,1,2,3,4,5,6,7
0,scaffold,source,type,start,stop,strand,info,ID
1,chr1,JGI,CDS,1381.0,2142.0,+,ID=gene_1,proteinId=2114025
2,chr1,JGI,CDS,2744.0,3343.0,-,ID=gene_2,proteinId=2293935
3,chr1,JGI,CDS,3344.0,3817.0,-,ID=gene_3,proteinId=2293936
4,chr1,JGI,CDS,5330.0,6385.0,-,ID=gene_4,proteinId=2051335


In [48]:
gff_path = 'Klac_Kmar_synonly.gff.txt'
fasta_path = 'Klac_Kmar.fsa.txt'
orthologs_path = '~/genomes_annotations/recovery_orthofinder_FINAL_wsacc.csv'

In [9]:
#gff = pd.read_csv(gff_path, sep='\t', header=None)[[0,1,2,3,4,6,7,8]]
thermo.columns = ['scaffold', 'source', 'type', 'start', 'stop', 'strand','info', 'ID']
#gff['ID'] = gff['info'].str.split(pat=';', expand=True)
#gff['species'] = gff['ID'].str[:2]

thermo.head()

,scaffold,source,type,start,stop,strand,info,ID
0,scaffold,source,type,start,stop,strand,info,ID
1,chr1,JGI,CDS,1381.0,2142.0,+,ID=gene_1,proteinId=2114025
2,chr1,JGI,CDS,2744.0,3343.0,-,ID=gene_2,proteinId=2293935
3,chr1,JGI,CDS,3344.0,3817.0,-,ID=gene_3,proteinId=2293936
4,chr1,JGI,CDS,5330.0,6385.0,-,ID=gene_4,proteinId=2051335


### Add promoter annotations

In [10]:
promoter_size = 1000

In [14]:
promoter_gff = pd.DataFrame(columns=thermo.columns)

for i, row in thermo.iterrows():
    
    promoter_row = row.copy()
    promoter_row['type'] = 'promoter'
    promoter_row['ID'] = row['ID']+'_promoter'
    
    if promoter_row['strand'] == '+':
        promoter_row['start'] = promoter_row['start'] - promoter_size
        promoter_row['stop'] = promoter_row['start']
        
    else:
        promoter_row['start'] = promoter_row['stop']
        promoter_row['stop'] = promoter_row['stop'] + promoter_size
    
    promoter_gff = promoter_gff.append(promoter_row)
    
    if i>100:
        break
    
combined_gff = combined_gff.append(promoter_gff).sort_values(['scaffold', 'start'])
combined_gff.head()

TypeError: can only concatenate str (not "int") to str

In [15]:
promoter_gff = pd.DataFrame(columns=thermo.columns)

# Ensure numeric types for arithmetic operations
thermo['start'] = pd.to_numeric(thermo['start'])
thermo['stop'] = pd.to_numeric(thermo['stop'])

for i, row in thermo.iterrows():
    promoter_row = row.copy()
    promoter_row['type'] = 'promoter'
    promoter_row['ID'] = row['ID'] + '_promoter'
    
    if promoter_row['strand'] == '+':
        promoter_row['start'] = promoter_row['start'] - promoter_size
        promoter_row['stop'] = promoter_row['start']
    else:
        promoter_row['start'] = promoter_row['stop']
        promoter_row['stop'] = promoter_row['stop'] + promoter_size
    
    promoter_gff = promoter_gff.append(promoter_row, ignore_index=True)
    
    if i > 100:
        break

combined_gff = combined_gff.append(promoter_gff, ignore_index=True).sort_values(['scaffold', 'start'])
combined_gff.head()


ValueError: Unable to parse string "start" at position 0

In [64]:
combined_gff['scaffold'] = combined_gff['scaffold'].apply(str)

In [65]:
combined_gff.to_csv('Klac_Kmar_CDS_and_promoters_orthologs_only.gff', sep='\t', index=None)